In [1]:
import time

import pandas as pd
import shap
import numpy as np
import tensorflow as tf
from pathlib import Path

from blume.table import table
from scipy.cluster.vq import whiten
from tqdm import tqdm
from tensorflow.keras.models import load_model
import matplotlib.pyplot as plt
from data_processing import pre_process_data
from obfuscation_functions import *
from util.custom_functions import replace_outliers_by_std, mean_std_analysis, replace_outliers_by_quartile
from shap_experiment import extract_shap, extract_shap_values, parse_shap_values_by_class


Num GPUs Available:  0


In [3]:
# datasets
audio_files_path = "./NNDatasets/audio"
gen_shap_df_path = './data/gen_shap_df.npy'
emo_shap_df_path = './data/emo_shap_df.npy'
print("Pre-processing audio files!")
x_train_emo_cnn, y_train_emo_encoded, x_test_emo_cnn, y_test_emo_encoded = pre_process_data(audio_files_path, get_emotion_label=True)
x_train_gen_cnn, y_train_gen_encoded, x_test_gen_cnn, y_test_gen_encoded = pre_process_data(audio_files_path, get_emotion_label=False)
print("Pre-processing audio files Complete!")

Pre-processing audio files!


100%|███████████████████████████████████████████| 6507/6507 [00:00<00:00, 2718089.45it/s]

Loading pre extracted raw audio from pkl file.


Loading pre extracted raw audio from pkl file successful.
Extracting mfccs from raw nd train audio data split.
Train data found. Loading from npy file.
Loading train successful.
Extracting mfccs from raw nd test audio data split.
Test data found. Loading from npy file.
Loading test data successful.


100%|███████████████████████████████████████████| 6507/6507 [00:00<00:00, 3852129.31it/s]


Loading pre extracted raw audio from pkl file.
Loading pre extracted raw audio from pkl file successful.
Extracting mfccs from raw nd train audio data split.
Train data found. Loading from npy file.
Loading train successful.
Extracting mfccs from raw nd test audio data split.
Test data found. Loading from npy file.
Loading test data successful.
Pre-processing audio files Complete!


In [6]:
# from keras import Sequential
from keras.layers import Conv2D, MaxPool2D
from tensorflow.keras import optimizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten, Dropout, Activation

model = Sequential()
model.add(Dense(128, input_shape=(40, )))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(16))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(2))
model.add(Activation('softmax'))
opt = optimizers.Adam()
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 128)               5248      
_________________________________________________________________
activation (Activation)      (None, 128)               0         
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                8256      
_________________________________________________________________
activation_1 (Activation)    (None, 64)                0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 16)                1

2022-01-25 14:31:44.841266: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [9]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x_train_emo_cnn = np.reshape(x_train_emo_cnn, (x_train_emo_cnn.shape[0], x_train_emo_cnn.shape[1]))
x_test_emo_cnn = np.reshape(x_test_emo_cnn, (x_test_emo_cnn.shape[0], x_test_emo_cnn.shape[1]))

x_train_emo_cnn_scaled = sc.fit_transform(x_train_emo_cnn)
x_test_emo_cnn_scaled = sc.transform(x_test_emo_cnn)

emo_train_dataset = tf.data.Dataset.from_tensor_slices((x_train_emo_cnn_scaled, y_train_emo_encoded))
gen_train_dataset = tf.data.Dataset.from_tensor_slices((x_train_emo_cnn_scaled, y_train_gen_encoded))

emo_test_dataset = tf.data.Dataset.from_tensor_slices((x_test_emo_cnn_scaled, y_test_emo_encoded))
gen_test_dataset = tf.data.Dataset.from_tensor_slices((x_test_gen_cnn, y_test_gen_encoded))

In [10]:
batch_size = 4
emo_train_dataset_batch = emo_train_dataset.batch(batch_size)
gen_train_dataset_batch = gen_train_dataset.batch(batch_size)

In [12]:
cnnhistory = model.fit(x_train_emo_cnn_scaled, y_train_gen_encoded, batch_size=4, epochs=100, validation_data=(x_test_emo_cnn_scaled, y_test_gen_encoded))
# Save the weights
#model.save(model_path)
    

Epoch 1/100
1135/1135 [==============================] - 1s 780us/step - loss: 0.0366 - accuracy: 0.9881 - val_loss: 0.0091 - val_accuracy: 0.9956
Epoch 2/100
1135/1135 [==============================] - 1s 731us/step - loss: 0.0216 - accuracy: 0.9929 - val_loss: 0.0063 - val_accuracy: 0.9982
Epoch 3/100
1135/1135 [==============================] - 1s 734us/step - loss: 0.0134 - accuracy: 0.9958 - val_loss: 0.0080 - val_accuracy: 0.9965
Epoch 4/100
1135/1135 [==============================] - 1s 733us/step - loss: 0.0172 - accuracy: 0.9941 - val_loss: 0.0034 - val_accuracy: 0.9991
Epoch 5/100
1135/1135 [==============================] - 1s 733us/step - loss: 0.0107 - accuracy: 0.9960 - val_loss: 0.0140 - val_accuracy: 0.9965
Epoch 6/100
1135/1135 [==============================] - 1s 739us/step - loss: 0.0104 - accuracy: 0.9963 - val_loss: 0.0043 - val_accuracy: 0.9991
Epoch 7/100
1135/1135 [==============================] - 1s 735us/step - loss: 0.0090 - accuracy: 0.9974 - val_loss: 0

1135/1135 [==============================] - 1s 781us/step - loss: 0.0030 - accuracy: 0.9991 - val_loss: 0.0011 - val_accuracy: 0.9991
Epoch 57/100
1135/1135 [==============================] - 1s 743us/step - loss: 0.0022 - accuracy: 0.9998 - val_loss: 0.0065 - val_accuracy: 0.9982
Epoch 58/100
1135/1135 [==============================] - 1s 736us/step - loss: 8.7526e-05 - accuracy: 1.0000 - val_loss: 0.0022 - val_accuracy: 0.9982
Epoch 59/100
1135/1135 [==============================] - 1s 731us/step - loss: 0.0062 - accuracy: 0.9987 - val_loss: 0.0083 - val_accuracy: 0.9982
Epoch 60/100
1135/1135 [==============================] - 1s 736us/step - loss: 6.0636e-04 - accuracy: 0.9998 - val_loss: 0.0011 - val_accuracy: 0.9991
Epoch 61/100
1135/1135 [==============================] - 1s 738us/step - loss: 0.0041 - accuracy: 0.9991 - val_loss: 0.0026 - val_accuracy: 0.9982
Epoch 62/100
1135/1135 [==============================] - 1s 737us/step - loss: 0.0052 - accuracy: 0.9989 - val_loss:

In [16]:
model.save("gmodel_checkpoint/gender_model_simple.h5")

In [17]:
model.evaluate(x_test_emo_cnn_scaled, y_test_gen_encoded)

36/36 [==============================] - 0s 826us/step - loss: 0.0060 - accuracy: 0.9982


[0.006006975192576647, 0.9982379078865051]